<a href="https://colab.research.google.com/github/adalves-ufabc/2025.Q3-PLN/blob/main/2025_Q3_PLN_AULA_18_Notebook_37.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

## **Agentes e RAG [LangChain]**
---



Uma das aplicações mais poderosas possibilitadas pelos LLMs são os chatbots sofisticados de perguntas e respostas (*Q&A*). Essas aplicações conseguem responder a perguntas sobre informações específicas de uma fonte. Elas utilizam uma técnica conhecida como Geração Aumentada por Recuperação (*RAG*, na sigla em inglês).

Este exemplo mostrará como construir uma aplicação simples de *Q&A* a partir de uma fonte de dados de texto não estruturado. Demonstraremos:

* Um agente RAG que executa buscas com uma ferramenta simples. Esta é uma boa implementação de propósito geral.

* Uma cadeia RAG de duas etapas que utiliza apenas uma única chamada de LLM por consulta. Este é um método rápido e eficaz para consultas simples.

**Conceitos**


Abordaremos os seguintes conceitos:

**Indexação**: um pipeline para ingerir dados de uma fonte e indexá-los. Isso geralmente ocorre em um processo separado.

**Recuperação e geração**: o processo RAG propriamente dito, que recebe a consulta do usuário em tempo de execução e recupera os dados relevantes do índice, passando-os em seguida para o modelo.

Após indexarmos nossos dados, usaremos um agente como nossa estrutura de orquestração para implementar as etapas de recuperação e geração.

#### **LangChain**

In [ ]:
#@title Instalando o pacote LangChain

!pip install -qU langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 15.6 MB/s eta 0:00:00


In [ ]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

1.0.5


In [ ]:
#@title Integração com o pacote da OpenAI

!pip install -qU langchain-openai

In [ ]:
#@title Definindo a chave da API da OpenAI

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
#@title Definindo a chave da API da OpenAI
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


#### **Exemplo**

Neste exemplo, criaremos um aplicativo que responde a perguntas sobre o conteúdo do site. O site específico que usaremos é a postagem do blog *LLM Powered Autonomous Agents*, de Lilian Weng, que nos permite fazer perguntas sobre o conteúdo da postagem.

Este exemplo requer as seguintes dependências:

In [ ]:
!pip install -qU langchain-text-splitters langchain-community langchain-core bs4

Defina um modelo:

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model="gpt-4.1", api_key=OPENAI_API_KEY)

Selecione um modelo de *embeddings*:

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=OPENAI_API_KEY)

Seleciona um *vector store*:

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

**Carregando documentos**

Primeiro, precisamos carregar o conteúdo da postagem do blog. Podemos usar **`DocumentLoaders`** para isso, que são objetos que carregam dados de uma fonte e retornam uma lista de objetos **`Document`**. Neste caso, usaremos o **`WebBaseLoader`**, que usa o `urllib` para carregar HTML de URLs da web e o `BeautifulSoup` para analisá-lo e convertê-lo em texto. Podemos personalizar a análise de HTML para texto passando parâmetros para o analisador `BeautifulSoup` por meio de `bs_kwargs`. Neste caso, apenas as tags HTML com as classes `post-content`, `post-title` ou `post-header` são relevantes, então removeremos todas as outras.

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43047


In [ ]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


**Dividindo documentos**

Nosso documento carregado tem mais de 42k caracteres, o que é muito longo para caber na janela de contexto de muitos modelos. Mesmo para aqueles modelos que conseguem exibir a postagem completa em sua janela de contexto, eles podem ter dificuldades para encontrar informações em entradas muito longas.

Para lidar com isso, dividiremos o **`Document`** em partes para *embedding*  e armazenamento vetorial (*vector storage*). Isso deve nos ajudar a recuperar apenas as partes mais relevantes da postagem do blog em tempo de execução.

Usaremos um `RecursiveCharacterTextSplitter`, que dividirá o documento recursivamente usando separadores comuns, como novas linhas, até que cada parte tenha o tamanho apropriado. Este é o divisor de texto recomendado para casos de uso de texto genérico.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


**Armazenando documentos**

Agora precisamos indexar nossos 63 blocos de texto para que possamos pesquisá-los em tempo de execução. Nossa abordagem é fazer o *`embed`* do conteúdo de cada divisão de documento e inserir essas incorporações em um armazenamento vetorial (*`vector store`*). Dada uma consulta de entrada, podemos então usar a busca vetorial para recuperar os documentos relevantes.

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['f9a542e6-3b5e-40b9-a1ef-59f02c9a3207', '2cd7c05a-6e48-412c-b7e8-0620006c93e4', '024fa0f8-3dec-4168-9b3e-2069aa3a41d3']


As aplicações `RAG` geralmente funcionam da seguinte maneira:

**Recuperação**: Dada uma entrada do usuário, as divisões relevantes são recuperadas do armazenamento usando um `Retriever`.

**Geração**: Um modelo produz uma resposta usando um prompt que inclui tanto a pergunta quanto os dados recuperados.

**Agentes RAG**

Uma formulação de uma aplicação `RAG` é como um agente simples com uma ferramenta que recupera informações. Podemos montar um agente `RAG` mínimo implementando uma ferramenta que encapsula nosso armazenamento vetorial:

In [ ]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

Com a ferramenta que temos, podemos construir o agente:

In [ ]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post."
    "Use the tool to help answer user queries."
)
agente = create_agent(modelo, tools, system_prompt=prompt)

Vamos testar isso. Criamos uma pergunta que normalmente exigiria uma sequência iterativa de etapas de recuperação para ser respondida:

In [ ]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agente.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_4APjHGJSJRWmGpgeFo2fFz82)
 Call ID: call_4APjHGJSJRWmGpgeFo2fFz82
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external cla

Observe que o agente:

1. Gera uma consulta para buscar um método padrão de decomposição de tarefas;

2. Ao receber a resposta, gera uma segunda consulta para buscar extensões comuns desse método;

3. Tendo recebido todo o contexto necessário, responde à pergunta.

**Cadeias RAG**

Na formulação RAG agentiva acima, permitimos que o LLM use seu critério para gerar uma chamada de ferramenta que ajude a responder às consultas do usuário.

Outra abordagem comum é uma cadeia de duas etapas, na qual sempre executamos uma busca (potencialmente usando a consulta bruta do usuário) e incorporamos o resultado como contexto para uma única consulta LLM. Isso resulta em uma única chamada de inferência por consulta, reduzindo a latência à custa da flexibilidade. Nessa abordagem, não chamamos mais o modelo em um loop, mas fazemos uma única passagem. Podemos implementar essa cadeia removendo ferramentas do agente e, em vez disso, incorporando a etapa de recuperação em um prompt personalizado:

In [ ]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agente = create_agent(modelo, tools=[], middleware=[prompt_with_context])

Vamos experimentar:

In [ ]:
query = "What is task decomposition?"
for step in agente.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================

Task decomposition is the process of breaking down a complex or complicated task into smaller, more manageable sub-tasks or steps. This makes it easier to solve large problems by handling each simple sub-task separately and often in sequence.

In AI and large language models (LLMs), task decomposition can be done using simple prompts (like "List the steps for doing XYZ"), through task-specific instructions (such as "Write a story outline" for novel writing), or with human input. Advanced methods like Chain of Thought (CoT) prompting encourage the model to "think step by step," while Tree of Thoughts creates a branching structure to explore multiple reasoning paths.

Another approach involves combining LLMs with external classical planners (using PDDL), where the model translates problems into plan

**Referência:**

> https://docs.langchain.com/oss/python/langchain/rag